<a href="https://colab.research.google.com/github/urieliram/tc_uc/blob/main/ETL_WEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lectura de archivos **CSV** de mercado eléctrico y conversión a formato **JSON** de Knueven2021



In [78]:
import pandas as pd
import numpy as np

In [80]:
hourslist=list(range(24))
# https://www.codegrepper.com/code-examples/python/how+to+read+csv+file+from+google+drive+on+google+colab+
path         = 'https://drive.google.com/uc?export=download&id=' 
URL_UNITRC   = 'https://drive.google.com/file/d/10wjxFWH1jKY5umDzVtfRJmbabNcOgtVI/view?usp=sharing'
URL_LIUNITRC = 'https://drive.google.com/file/d/11MTOqeVswVb2ZnfWsS2fYI5-4f6jR_WE/view?usp=sharing'

UNITRC   = pd.read_csv(path + URL_UNITRC.split('/')[-2],  usecols=[0,1,4],  names=['CLVUNI', 'TYPE', 'NODE']) ## Datos de unidades de rango continuo) .isna().sum()
LIUNITRC = pd.read_csv(path + URL_LIUNITRC.split('/')[-2],usecols=hourslist,names=['LIM1','LIM2','LIM3','LIM4','LIM5','LIM6','LIM7','LIM8','LIM9','LIM10','LIM11','LIM12','LIM13','LIM14','LIM15','LIM16','LIM17','LIM18','LIM19','LIM20','LIM21','LIM22','LIM23','LIM24']) ## Límite inferior de generación de unidades de rango continuo

In [81]:
print(LIUNITRC)

      LIM1   LIM2   LIM3   LIM4   LIM5   LIM6   LIM7   LIM8   LIM9  LIM10  \
0      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
..     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
250    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
251    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
252  180.0  180.0  180.0  180.0  180.0  180.0  180.0  180.0  180.0  180.0   
253   60.0   60.0   60.0   60.0   60.0   60.0   60.0   60.0   60.0   60.0   
254   60.0   60.0   60.0   60.0   60.0   60.0   60.0   60.0   60.0   60.0   

     ...  LIM15  LIM16  LIM17  LIM18  LIM19  LIM20  LIM21  LIM22  LIM23  LI

"thermal_generators":{
      "G1":{
         "must_run":0,
         "power_output_minimum":80,
         "power_output_maximum":300.0,
         "ramp_up_limit":50,
         "ramp_down_limit":30,
         "ramp_startup_limit":100,
         "ramp_shutdown_limit":80,
         "time_up_minimum":3,
         "time_down_minimum":2,
         "power_output_t0":120,
         "unit_on_t0":1,
         "time_up_t0":1,
         "time_down_t0":0,
         "startup"
         }